# Import packages

In [1]:
# Import package
import numpy as np
import pandas as pd

# Cleaning data

In [2]:
df = pd.read_csv('marx-geo.csv', header=None, sep='\n') #Read csv and separate by line
df = df[0].str.split(',', expand=True) #Split first row(header)


In [3]:
new_header = df.iloc[0] #Grab the first row for the header
df = df[1:] #Take the data less the header row
df.columns = new_header #Set the header row as the df header
df.rename(columns = {'NaN':'real_longitude'}, inplace = True)
df.columns = df.columns[:15].tolist() + ['real_longitude'] #Select the column from first to the next to the last column, and give the wrong data column a header

In [4]:
# Editing the correct latitude and longitude to the right column

for i in range(len(df)):
    if df.iloc[i, 12] == 'DC':
        df.iloc[i, 12] = df.iloc[i, 13] #Replace with the correct postcode
        df.iloc[i, 13] = df.iloc[i, 14] #Replace with the correct latitude
        df.iloc[i, 14] = df.iloc[i, 15] #Replace with the correct longitude

df.drop(['real_longitude'], axis=1, inplace = True) #Drop the real_longitude column

In [5]:
# Some data in 'country' column is missing, but has the data in 'cc' column. Adding the missing data by the data in 'cc' column
df['country'] = df.groupby('cc')['country'].transform('first') # Group by 'cc', and get the first none NAN value

# Fill the missing data in column 'country' with the reference data(latitude, longitude)
df['country'] = df.groupby(['latitude', 'longitude'])['country'].transform('first')

# Fill the missing data in column 'cc' with the reference data(country)
df['cc'] = df.groupby('country')['cc'].transform('first')

# Mark the unknow value in 'country' column
df['country'].replace(to_replace=[''], value="unknown", inplace=True)

In [6]:
df.head()

,datetime,host,src,proto,type,spt,dpt,srcstr,cc,country,locale,localeabbr,postalcode,latitude,longitude
1,2013-03-03 21:53:59,groucho-oregon,1032051418,TCP,,6000,1433,61.131.218.218,CN,China,Jiangxi Sheng,36,,28.55,115.9333
2,2013-03-03 21:57:01,groucho-oregon,1347834426,UDP,,5270,5060,80.86.82.58,DE,Germany,,,,51.0,9.0
3,2013-03-03 21:58:10,groucho-oregon,2947856490,TCP,,2489,1080,175.180.184.106,TW,Taiwan,Taipei,,,25.0392,121.525
4,2013-03-03 21:58:09,groucho-us-east,841842716,UDP,,43235,1900,50.45.128.28,US,United States,Oregon,OR,97124,45.5848,-122.9117
5,2013-03-03 21:58:20,groucho-singapore,3587648279,TCP,,56577,80,213.215.43.23,FR,France,,,,48.86,2.35


# Start analysis

## Analysis by country

In [7]:
#Couunts attack times for each country, and reset index to make it as a df
country_view = df["country"].value_counts().reset_index()

# Rename the header
country_view.rename(columns={'index':'country', 'country':'attack times'}, inplace = True)

# Add a column shows total percentage of the attacks
country_view['attack times by total %'] = (country_view["attack times"] / len(df)).map("{:.2%}".format)

# Reset the index to country
#country_view.set_index('country', inplace = True)

In [8]:
# Top 15 country
country_view.head(15)

,country,attack times,attack times by total %
0,China,191404,42.39%
1,United States,90005,19.93%
2,Japan,17204,3.81%
3,Iran,13042,2.89%
4,Taiwan,12150,2.69%
5,Netherlands,10739,2.38%
6,India,9418,2.09%
7,South Korea,9316,2.06%
8,Vietnam,7826,1.73%
9,Russia,7211,1.60%


## Analysis by latitude and longitude (Location)

In [9]:
# Set the group_by parameter ('latitude', 'longitude', "country")
by_latitude_longitude = df.groupby(['latitude', 'longitude', "country"])

In [10]:
#Couunts attack times for each location, and reset index to makes it as a df
location_view = by_latitude_longitude['datetime'].count().sort_values(ascending=False).reset_index()

# Rename the header
location_view.rename(columns={'datetime':'attack times'}, inplace = True)

# Add a column shows total percentage of the attacks
location_view['attack times by total %'] = (location_view["attack times"] / len(df)).map("{:.2%}".format)

# Add a column shows total percentage of its country
location_view = pd.merge(location_view, country_view, how = 'left', on = "country") # Merge two df
location_view['attack times by its country %'] = (location_view['attack times_x'] / location_view['attack times_y']).map("{:.2%}".format) # Add a column of percentage of its country
location_view.drop(['attack times_y', 'attack times by total %_y'], axis=1, inplace = True) # Drop the redundant columns
location_view.rename(columns={'attack times_x':'attack times', 'attack times by total %_x':'attack times by total %'}, inplace = True) # Change the header

# Reset the index to location
location_view.set_index(['latitude', 'longitude'], inplace = True)

In [11]:
# Top first 15 location
location_view.head(15)

,,country,attack times,attack times by total %,attack times by its country %
latitude,longitude,,,,
39.9289,116.3883,China,33564,7.43%,17.54%
32.0617,118.7778,China,32832,7.27%,17.15%
41.7922,123.4328,China,20956,4.64%,10.95%
35.0,105.0,China,13396,2.97%,7.00%
31.8639,117.2808,China,13324,2.95%,6.96%
38.08,46.2919,Iran,11122,2.46%,85.28%
35.69,139.69,Japan,10715,2.37%,62.28%
25.0392,121.525,Taiwan,9760,2.16%,80.33%
34.0115,-117.8535,United States,9305,2.06%,10.34%


In [12]:
# Check how many country's attacks are made by same person or group (same geographic locatio)

# Set the filter for percentage is greater than 80%
by_country_pct = location_view['attack times by its country %'].str.strip('%').astype('float') > 80

# Show the result of the percentage is greater than 80% location. There are 84 locations.
location_view[by_country_pct]

,,country,attack times,attack times by total %,attack times by its country %
latitude,longitude,,,,
38.08,46.2919,Iran,11122,2.46%,85.28%
25.0392,121.525,Taiwan,9760,2.16%,80.33%
16.0,106.0,Vietnam,6338,1.40%,80.99%
,,unknown,3428,0.76%,87.36%
62.0,15.0,Sweden,2820,0.62%,95.14%
...,...,...,...,...,...
43.9333,12.4667,San Marino,2,0.00%,100.00%
-22.0,17.0,Namibia,1,0.00%,100.00%
32.2942,-64.7839,Bermuda,1,0.00%,100.00%


In [13]:
# Attacks made by these countries are mmostly from one location (>80% of total attacks in this country)
location_view[by_country_pct]['country']

latitude  longitude
38.08     46.2919            Iran
25.0392   121.525          Taiwan
16.0      106.0           Vietnam
                          unknown
62.0      15.0             Sweden
                          ...    
43.9333   12.4667      San Marino
-22.0     17.0            Namibia
32.2942   -64.7839        Bermuda
13.1667   -59.5333       Barbados
16.0      -24.0        Cape Verde
Name: country, Length: 84, dtype: object

## Analysis with only top 5 country

In [14]:
# Find the top 5 country
top_country = country_view['country'].head(5).tolist()

# Create a df for top 5 country
top_country_view = df[df['country'].isin(top_country)]

# Reset the index
top_country_view.reset_index(drop=True, inplace = True)

In [15]:
top_country_view.head()

,datetime,host,src,proto,type,spt,dpt,srcstr,cc,country,locale,localeabbr,postalcode,latitude,longitude
0,2013-03-03 21:53:59,groucho-oregon,1032051418,TCP,,6000,1433,61.131.218.218,CN,China,Jiangxi Sheng,36,,28.55,115.9333
1,2013-03-03 21:58:10,groucho-oregon,2947856490,TCP,,2489,1080,175.180.184.106,TW,Taiwan,Taipei,,,25.0392,121.525
2,2013-03-03 21:58:09,groucho-us-east,841842716,UDP,,43235,1900,50.45.128.28,US,United States,Oregon,OR,97124,45.5848,-122.9117
3,2013-03-03 21:58:41,groucho-tokyo,3323217250,TCP,,32628,2323,198.20.69.98,US,United States,Illinois,IL,60661,41.8825,-87.6441
4,2013-03-03 21:59:36,groucho-oregon,3730416887,TCP,,6000,1433,222.89.164.247,CN,China,Henan Sheng,41,,34.6836,113.5325


## Analysis with host

In [16]:
#Couunts attack times for each host, and reset index to make it as a df
host_view = df["host"].value_counts().reset_index()

# Rename the header
host_view.rename(columns={'index':'host', 'host':'attack times'}, inplace = True)

# Add a column shows total percentage of the attacks
host_view['attack times by total %'] = (host_view["attack times"] / len(df)).map("{:.2%}".format)

# Reset the index to host
host_view.set_index('host', inplace = True)

In [17]:
host_view

,attack times,attack times by total %
host,,
groucho-tokyo,126189,27.94%
groucho-oregon,94076,20.83%
groucho-singapore,78151,17.31%
groucho-us-east,31779,7.04%
groucho-norcal,24566,5.44%
groucho-sydney,24456,5.42%
groucho-sa,24316,5.38%
zeppo-norcal,24094,5.34%
groucho-eu,23954,5.30%


## Analysis with network protocol

In [18]:
# See which network protocol has higher volume of traffic


#Couunts attack times for each network protocol, and reset index to make it as a df
proto_view = df['proto'].value_counts().reset_index()

# Rename the header
proto_view.rename(columns={'index':'proto', 'proto':'attack times'}, inplace = True)

# Add a column shows total percentage of the attacks
proto_view['attack times by total %'] = (proto_view["attack times"] / len(df)).map("{:.2%}".format)

# Drop the row that doesn't have network protocol in data
#proto_view = proto_view[proto_view['proto'] != '']

# Reset the index to network protocol
proto_view.set_index('proto', inplace = True)

In [19]:
proto_view

,attack times,attack times by total %
proto,,
TCP,327991,72.63%
UDP,78779,17.45%
ICMP,44811,9.92%


## Analysis with destination port

In [20]:
# Import the ports data from https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers
ports = pd.read_csv('ports.csv', header = 0, usecols = ['Port', 'TCP', 'UDP', 'Description']) #Read csv 

In [21]:
ports.head()

,Port,TCP,UDP,Description
0,1024,Reserved,Reserved,Reserved
1,1027,Reserved,NaN,Reserved
2,NaN,NaN,Yes,Native IPv6 behind IPv4-to-IPv4 NAT Customer P...
3,1028,Reserved[2],NaN,NaN
4,1029,Unofficial,NaN,Microsoft DCOM services


In [22]:
#Couunts attack times for each destination port, and reset index to make it as a df
dpt_view = df["dpt"].value_counts().reset_index()

# Rename the header
dpt_view.rename(columns={'index':'dpt', 'dpt':'attack times'}, inplace = True)

# Add a column shows total percentage of the attacks
dpt_view['attack times by total %'] = (dpt_view["attack times"] / len(df)).map("{:.2%}".format)

# Drop the row that doesn't have destination port in data
dpt_view = dpt_view[dpt_view['dpt'] != '']

# Reset the index to dpt
dpt_view.set_index('dpt', inplace = True)

In [23]:
# Top 5 ports being attacked
dpt_view.head()

,attack times,attack times by total %
dpt,,
1433,109425,24.23%
445,40611,8.99%
3389,30077,6.66%
80,19575,4.33%
56338,18196,4.03%


In [24]:
# See what is port 1433
ports[ports['Port'] == '1433']

,Port,TCP,UDP,Description
45,1433,Yes,Yes,Microsoft SQL Server database management syste...


## Analysis with sorce port

In [25]:
#Couunts attack times for each sorce port, and reset index to make it as a df
spt_view = df['spt'].value_counts().reset_index()

# Rename the header
spt_view.rename(columns={'index':'spt', 'spt':'attack times'}, inplace = True)

# Add a column shows total percentage of the attacks
spt_view['attack times by total %'] = (spt_view["attack times"] / len(df)).map("{:.2%}".format)

# Drop the row that doesn't have sorce port in data
#proto_view = proto_view[proto_view['proto'] != '']

# Reset the index to spt
#proto_view.set_index('proto', inplace = True)

In [26]:
# Top 5 ports attacked from
spt_view.head()

,spt,attack times,attack times by total %
0,6000,154133,34.13%
1,,44811,9.92%
2,25416,18195,4.03%
3,10100,11116,2.46%
4,4445,4716,1.04%


In [28]:
# See what is port 6000
print(ports[ports['Port'].str.contains('6000', na=False)])

          Port         TCP  UDP  \
489  6000–6063         Yes  Yes   
891      16000  Unofficial  NaN   
975      26000         Yes  Yes   

                                           Description  
489  X11—used between an X client and server over t...  
891  Oracle WebCenter Content: Imaging (formerly kn...  
975                         id Software's Quake server  


In [286]:
# For donloading the df
#df.to_csv(r'C:\Users\gogoorange\OneDrive\桌面\Monad Data Analyst Exercise\cleaned_data2.csv', index=False, header = True)

In [117]:
# For donloading the df
#top_country_view.to_csv(r'C:\Users\gogoorange\OneDrive\桌面\Monad Data Analyst Exercise\top_5_country.csv', index=False, header = True)